In [113]:
import sqlite3, random, pathlib
from search import search_library
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from pydantic import BaseModel, Field, ValidationError
import os
from openai import OpenAI
from pydantic import BaseModel
import json
import time
import random
from helpers.sentence_splitter import split_sentences

In [121]:
def ask(sys_msg, usr_msg):
    resp = client.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": sys_msg},
            {"role": "user", "content": usr_msg}
        ]
    )
    return resp.choices[0].message.content

def print_obj(obj, i=1, params=["agent", "interaction", "space", "time", "causal link", "physical law"]):
    print(f"{i} - ")
    for param in params:
        print(f"{param}: {obj[param]}")
    

def parse_func_resp(resp, obj_name="element", params=["name", "text"], print="True"):
    args_raw = resp.choices[0].message.tool_calls[0].function.arguments
    raws = json.loads(args_raw)[obj_name]

    obj_lst = []
    for i, obj in enumerate(raws, 1):
        if print:
            pass
            #print_obj(obj, i, params)
        obj_lst.append(obj)
    
    return obj_lst

def elements_to_string(dims, label="ELEMENTS"):
    s = ""
    for dim, elems in dims.items():
        s += f"{dim}\n{label}:\n"
        for elem in elems:
            s += elem + "\n"
        s += "\n"

    return s

def sample_text(text, x):
    max_start = max(0, len(text) - x)
    random_start = random.randint(0, max_start)
    excerpt = text[random_start:random_start + x]
    return excerpt

def ask_func(sys_msg, usr_msg, FUNC):
    resp = client.chat.completions.create(
            model=OPENAI_MODEL,
            messages=[{"role":"system","content":sys_msg}, {"role":"user","content":usr_msg}],
            tools=[FUNC], tool_choice="auto", temperature=0.5
            )
    return resp

def sample_elems(elements, per_dim, per_elem):
    total_dim = len(elements)
    ndims = min(int(per_dim * total_dim), total_dim)

    # Sample dimension indices
    dimension_indices = random.sample(range(total_dim), ndims)

    sample_elems = {}

    for di in dimension_indices:
        dimension_name = list(elements.keys())[di]
        dimension_elems = list(elements.values())[di]

        total_elems = len(dimension_elems)
        nelems = min(int(per_elem * total_elems), total_elems)

        # Sample element indices from this dimension
        element_indices = random.sample(range(total_elems), nelems)
        sample_elems[dimension_name] = [dimension_elems[ei] for ei in element_indices]

    return sample_elems

def clean_elements(elements_raw):
    elements = {}
    for element_raw in elements_raw:
        k = element_raw["name"]
        v = element_raw["text"]
        element = (k, [item.strip() for item in v.split('\n') if item.strip()])
        elements[element[0]] = element[1]
    return elements
    



load_dotenv()
client = OpenAI()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o")

DB_FILE = "library.sqlite"
MAX_RESULTS = 100

In [133]:
class story(BaseModel):
    agent: str
    interaction: str
    space: str
    time: str
    link: str
    law: str

    
SYS_PROMPT_ELEMENTS = """
My goal is a description of the mechanical forces driving the development of the hindgut and an understanding of how the overexpression of myosin 1c drives the inversion of LR asymmetry 
during development. I want to write a complete story of hindgut development incorporating mechanical force as central players in this story, 
then create a complete story of hindgut development under Myo1C OE, the generation of LR inversion from the role of Myo1C to the hindgut rotation. 
The following text is from a paper on hindgut morphogenesis.
Extract a list of agents, interactions, spatial compartments, developmental stages, causal chains, and physical constraints involved in hindgut morphogenesis and the effect of myosin 1c overexpression on LR inversion.
Structure the output as a story scaffold:
Agents: ...
Interactions: ...
Spaces: ...
Time: ...
Causal links: ...
Physical laws/constraints: ...
"""


SYS_PROMPT_DIMENSIONS = """
You are helping build a complete narrative of hindgut morphogenesis in Drosophila, with special attention to the role of mechanical forces - driving the rotation and elongation of the hindgut - and the overexpression of myosin 1c (Myo1C) in left-right (LR) inversion.

Your task is not to tell the story, but to define what would be required to tell a **complete and comprehensive story** for this set of story elements.

- Break down these elements into their essential **story dimensions** or **sub-components**.
- Think about biological scale (molecular, cellular, tissue), space, time, dynamics, agents, interactions, functions, and unknowns.
- Consider how these elements might relate to:
  - other parts of the hindgut
  - the mechanical forces shaping the tissue
  - the overexpression of Myo1C and its effects on LR inversion

Output a structured list of **story dimensions** that should be explored in order to fully develop these elements. If helpful, include example questions or variables within each dimension.

Format:

**1. Dimension Name**  
- Description of what this dimension involves.  
- Example questions or aspects to explore.

(Repeat as needed for all relevant dimensions)
"""


SYS_PROMPT_PARSE_ELEMENTS = ("Read the following text and separate each element of the suggested story framework into a object with the name and text.\n"
        "You should return a list of objects containing each story element in a given structure.\n"
        "Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.\n"
        "Treat every line that begins with a number followed by “.” and a space (e.g. “1. ”) or **Element** as the NAME of a new element."
        "Capture all following lines up to—but not including—the next such line as the TEXT for that element.")


SYS_PROMPT_STORYTELLING = """
You are constructing a complete and mechanistically rich **narrative of Drosophila hindgut morphogenesis**, spanning from **stage 11 to stage 15**, with a particular focus on the role of **mechanical forces** in driving **left-right (LR) asymmetric rotation** and **anterior-posterior elongation** of the gut.

This narrative must:
- Progress through **a clear beginning, middle, and end**, anchored to specific developmental stages.
- Integrate biological elements (agents, interactions, causal links, physical laws, etc.) into a **continuous developmental story**.
- Be organized by **stages of development**, but deeply informed by **conceptual dimensions** like spatial organization, tissue morphogenesis, molecular interactions, temporal regulation, and unknowns.

---

### 🔬 BIOLOGICAL ELEMENTS TO INTEGRATE
These are grouped by type and must be woven into the narrative. Explain how they emerge, interact, or evolve across time, and how they contribute to hindgut development and LR asymmetry.

{elements_text}

---

### 🧭 DIMENSIONS OF STORY STRUCTURE
Use the following conceptual dimensions to guide the exploration of how the elements interact and function over time. Within each phase (beginning, middle, end), consider these dimensions to reveal biological meaning and causal relationships.

{dimensions_text}

---

### 📘 INSTRUCTIONS FOR STORYTELLING

- Begin at **stage 11**, describing the initial state of the hindgut and its relevant cellular, molecular, and tissue-level configurations.
- In the **middle**, describe the transformation from stage 11 to stage 14: how elements and dimensions evolve, interact, and drive morphogenetic change.
- Conclude at **stage 15**, describing the final morphology, internal organization, and functional integration of the hindgut.

At each stage:
- Use the **dimensions** as lenses to explore how the elements operate and change.
- Show how mechanical forces, molecular regulators, spatial constraints, and timing orchestrate morphogenesis.
- Ensure developmental sequencing is causally and spatially coherent.

The story should flow like a scientific narrative that unfolds through time, with mechanistic insight and biological precision.

---

### 🔚 ENDING THE STORY

Conclude with:
- A synthesis of how the initial configuration transformed into the final morphology.
- Key causal mechanisms and checkpoints.

"""

SYS_PROMPT_STORYTELLING2 = """
You are rewriting a complete developmental narrative of **Drosophila hindgut morphogenesis from stage 11 to stage 15**, under the condition that **Myosin 1C (Myo1C) is overexpressed across the brachyenteron domain**, leading to **inversion of left-right (LR) asymmetry**.

The original story below describes normal hindgut development through molecular, cellular, tissue, and mechanical dimensions.

---

### 📖 Original Wild-Type Narrative:
{wt_story}

---

### 🎯 New Objective: Mutation-Driven Story of LR Inversion

You must now **retell the full developmental story** from stage 11 to stage 15, describing how **Myo1C overexpression disrupts, rewires, or distorts the normal developmental trajectory**.

Structure the story as a **temporal narrative** with:

---

### I. **BEGINNING (Stage 11)**  
Describe the state of the hindgut at stage 11 *under Myo1C OE*. Which molecular players, spatial structures, and physical forces are already altered? What remains similar to wild-type?

---

### II. **MIDDLE (Transition through Stage 12–14)**  
Trace the dynamic changes in morphology, mechanical force propagation, cellular behaviors, gene expression, and spatial orientation.  
- What **new causal chains** emerge?
- How are existing **feedbacks, constraints, or symmetry cues disrupted**?
- Which **interactions or dimensions break down or invert**?
- Which **structures and dynamics remain the unaffected**?

---

### III. **END (Stage 15 Outcome)**  
What is the **final anatomical and functional state** of the hindgut under Myo1C OE?  
- Describe the **phenotypic result**, such as LR inversion or malformed rotation.
- Explain how the cascade of altered elements leads to this specific outcome.

---

### 📘 Use the following conceptual dimensions as lenses throughout the narrative:
{dimensions_text}

These should guide your mechanistic reasoning within each time window. Explain how each dimension is impacted by the overexpression of Myo1C and contributes to the inverted outcome.

---

### 🔬 Reintegrate Relevant Elements:
{elements_text}

Explicitly connect these elements to the evolving story. Focus on **mechanical, molecular, spatial, and causal specificity**.

---

### ✅ Guidelines:
- Maintain a **scientific tone** grounded in known biology and plausible hypotheses.
- Emphasize **mechanistic detail**: how does Myo1C overexpression perturb cytoskeletal tension, spatial polarity, adhesion, or developmental timing?
- If Myo1C dominates certain molecular interactions or creates feedback loops, show how these propagate to higher levels.
- Be consistent with developmental stage timing and spatial logic.
- Where evidence is incomplete, **propose testable hypotheses**.

---

### 🔚 Conclusion:
Synthesize the full inversion narrative:
- What are the **key causal disruptions**?
- How does Myo1C act as the initiating factor?

"""


ELEM_FUNC = {
    "type": "function",
    "function": {
        "name": "element_function",
        "description": "Read the following text and separate each element of the suggested story framework into a object with the name and text. \
        You should return a list of objects containing each story element in a given structure. \
        Do not add or remove text; your job is to parse and deconstruct the text string into the given structure.",
        "parameters": {
            "type": "object",
            "properties": {
                "element": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": { "type": "string" },
                            "text": { "type": "string" }
                        },
                        "required": ["name", "text"]
                    }
                }
            },
            "required": ["element"]
        }
    }
}

In [10]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    print(title, len(text))

Active Tension Network Model in Epithelial Tissues 32334
The Geometric Basis of Epithelial Convergent Extension 188479
Generating Active T1 Transitions through Mechanochemical Feedback 117183


In [9]:
KEYWORDS = ["active tension network", "epithelial"]  # list of words
search_results = search_library(DB_FILE, KEYWORDS, MAX_RESULTS, match_all=True)
# papers = {}
for r in search_results:
    title = r['title']
    text = r['text']
    papers[title] = text


rem = [None, "Fundamental Neuroscience (3rd edition)","Harrison's Principles of Internal Medicine 15th Edition", "Developmental Biology, 7th Edition"]
for r in rem:
    if r in papers:
        del papers[r]


for title in list(papers.keys()):
    print(title)

The Drosophila actin nucleator DAAM is essential for left-right asymmetry
Myosin1D is an evolutionarily conserved regulator of animal left–right asymmetry
An unconventional myosin in Drosophila reverses the default handedness in visceral organs
Chiral cell sliding drives left-right asymmetric organ twisting
Cell Chirality Drives Left-Right Asymmetric Morphogenesis
Chirality in Planar Cell Shape Contributes to Left-Right Asymmetric Epithelial Morphogenesis
Vertebrate myosin 1d regulates left–right organizer morphogenesis and laterality
Distinct Cellular and Junctional Dynamics Independently Regulate the Rotation and Elongation of the Embryonic Gut in Drosophila
Understanding Laterality Disorders and the Left-Right Organizer: Insights from Zebrafish
Generating Active T1 Transitions through Mechanochemical Feedback
Left–right Myosin-Is, Myosin1C, and Myosin1D exhibit distinct single molecule behaviors on the plasma membrane of Drosophila macrophages
It Takes Guts: The Drosophila Hindgut a

In [131]:
for n in range(1):
    print(n)

0


In [132]:
stories = {}

for n in range(1): # range(len(papers)):
    title, text = list(papers.keys())[n], list(papers.values())[n]
    excerpt = sample_text(text, 10000)

    sys_msg = SYS_PROMPT_ELEMENTS
    user_msg = f"TEXT EXCERPT:\n{excerpt}"
    elements_resp = ask(sys_msg, user_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY FRAMEWORK:\n{elements_resp}"
    elements_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    elements_raw = parse_func_resp(elements_func, "element", ["name", "text"], "False")
    elements = clean_elements(elements_raw)

    per_dim, per_elem = 1, 1   # Number of elements per dimension to sample
    elements_sample = sample_elems(elements, per_dim, per_elem)
    elements_text = elements_to_string(elements_sample, "ELEMENTS")

    # GET DIMENSIONS
    sys_msg = SYS_PROMPT_DIMENSIONS
    usr_msg = f"Here are the story elements:\n\n{elements_text}"
    dimensions_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_PARSE_ELEMENTS
    usr_msg = f"STORY DIMENSIONS:\n{dimensions_resp}"
    dimensions_func = ask_func(sys_msg, usr_msg, ELEM_FUNC)

    dimensions_raw = parse_func_resp(dimensions_func, "element", ["name", "text"], "False")
    dimensions = clean_elements(dimensions_raw)

    # Configuration
    per_dim, per_elem = 1, 1 
    dimensions_sample = sample_elems(dimensions, per_dim, per_elem)
    dimensions_text = elements_to_string(dimensions_sample, "DIMENSIONS")

    sys_msg = SYS_PROMPT_STORYTELLING.format(elements_text=elements_text, dimensions_text=dimensions_text)
    usr_msg = "Please generate the complete, structured narrative based on the elements and dimensions provided."
    wt_resp = ask(sys_msg, usr_msg)

    sys_msg = SYS_PROMPT_STORYTELLING2.format(wt_story=wt_resp, elements_text=elements_text, dimensions_text=dimensions_text)
    usr_msg = "Please rewrite the original story to reflect the overexpression of Myosin 1C (Myo1C) across the brachyenteron domain. Show how this perturbation leads to inversion of left-right asymmetry in the Drosophila hindgut by altering the behavior of affected molecular, cellular, and tissue-level elements. Maintain the original story structure and focus on causal, mechanistic explanations."
    myo_resp = ask(sys_msg, usr_msg)

    stories[title] = {
        "elements": elements_sample,
        "dimensions": dimensions_sample,
        "wt_story": wt_resp,
        "myo_story": myo_resp
    }


    print(title, "\n\n")
    print(wt_resp, "\n\n")
    print(myo_resp, "\n\n\n")

The Drosophila actin nucleator DAAM is essential for left-right asymmetry 


### **The Story of Drosophila Hindgut Morphogenesis: From Beginnings to LR Asymmetry**

---

### **Stage 11: The Beginning**

At stage 11, the Drosophila embryo is poised for a critical sequence of developmental events leading to the morphogenesis of the hindgut. Initially, the hindgut tissue comprises two main domains: the H1 and H2 regions. Spatial organization within these regions relies heavily on the precise localization of key proteins, notably Myo1D and DAAM, involved in cytoskeletal dynamics and mechanical force generation.

#### **Molecular Interactions and Cellular Dynamics**
During this early stage, DAAM begins its critical role in cytoskeletal structuring, particularly actin polymerization. Actin filaments, organized by DAAM, form the structural backbone necessary for subsequent cellular shape changes and motility. At the molecular level, Myo1D interacts with DAAM, as suggested by co-immunoprecipit

In [134]:
SYSTEM_PROMPT_REVIEW = """"
You are reviewing a developmental biology narrative of Drosophila hindgut morphogenesis. Your task is to:
- Extract all statements that are **directly supported by experimental data** (including references if available).
- Extract all statements that are **plausible but hypothetical or inferred**.
- Clearly distinguish these two categories and highlight any assumptions.

Format:
- Data-Supported: [...]
- Hypotheses: [...]
- Assumptions: [...]
"""
sys_msg = SYSTEM_PROMPT_REVIEW
usr_msg = f"Here are the narrative of hindgut morphogenesis:\n\n{wt_resp}"
review = ask(sys_msg, usr_msg)
print(wt_resp, "\n\nREIVEW:\n")
print(review)

### **The Story of Drosophila Hindgut Morphogenesis: From Beginnings to LR Asymmetry**

---

### **Stage 11: The Beginning**

At stage 11, the Drosophila embryo is poised for a critical sequence of developmental events leading to the morphogenesis of the hindgut. Initially, the hindgut tissue comprises two main domains: the H1 and H2 regions. Spatial organization within these regions relies heavily on the precise localization of key proteins, notably Myo1D and DAAM, involved in cytoskeletal dynamics and mechanical force generation.

#### **Molecular Interactions and Cellular Dynamics**
During this early stage, DAAM begins its critical role in cytoskeletal structuring, particularly actin polymerization. Actin filaments, organized by DAAM, form the structural backbone necessary for subsequent cellular shape changes and motility. At the molecular level, Myo1D interacts with DAAM, as suggested by co-immunoprecipitation studies, indicating a synergistic role in orchestrating the actin netwo